# Coherence tests

[L. Blackburn, Sep 2018]

The purpose of this test is to check the degree of phase coherence on different baslines by calculating the amplitude loss using different coherent integration timescales. Poor phase coherence can be a result of low signal-to-noise and poor atmosphere, or an issue with ad-hoc phase correction.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
from collections import OrderedDict
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.ticker import MultipleLocator
import seaborn as sns
sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']
import sys
nb_stdout = sys.stdout # grab for later
def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()
def tightx(): plt.autoscale(enable=True, axis='x', tight=True)
def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')
def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])
# pd.options.display.float_format = '{:,.6f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))
import string

In [ ]:
os.getcwd()

In [ ]:
# define and load data
alist0 = 'alist.v6.2s.avg'
alist1 = 'alist.v6.30s.avg'
# stage = '5.+close'
# datadir = '/data/2017-april/ce/er4test/hops-lo/%s/data/' % stage
datadir = os.environ['DATADIR'] + '/'
a0 = util.noauto(hops.read_alist(datadir + alist0))
a1 = util.noauto(hops.read_alist(datadir + alist1))
util.fix(a0)
util.add_days(a0)
util.add_path(a0)
util.add_scanno(a0)
util.fix(a1)
util.add_days(a1)
util.add_path(a1)
util.add_scanno(a1) 

In [ ]:
a0 = a0[a0.polarization.isin({'RR', 'LL'})].set_index(
    "expt_no source scan_id baseline polarization".split()).groupby(
    'expt_no source scan_id baseline'.split()).mean()
a1 = a1[a1.polarization.isin({'RR', 'LL'})].set_index(
    "expt_no source scan_id baseline polarization".split()).groupby(
    'expt_no source scan_id baseline'.split()).mean()
a0['snr1'] = a1.snr
a0['amp1'] = a1.amp
a0['coh'] = a1.amp / a0.amp
a = a0.reset_index().dropna()

In [ ]:
plt.loglog(a.snr, a.coh, '.', ms=1)
plt.axvline(7, color='k', ls='--', lw=2, alpha=0.25)
plt.axhline(1, color='k', ls='--', lw=2, alpha=0.25)
plt.ylim(.2, 2)
plt.xlim(1, None)
plt.xlabel('2s SNR')
plt.ylabel('30s relative amplitude')
wide(8, 4)

In [ ]:
# data filters
a = a[(a.snr > 7.) & ~a.baseline.isin({'RS', 'SR'}) & ~a.baseline.str.contains('R')]

In [ ]:
# expt_no boundary ticker marks vs scan
elines = a.scan_no.sort_values().values[np.nonzero(np.diff(a.expt_no.sort_values()) > 0)[0]] + 0.5

In [ ]:
# show all output in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
bls = sorted(set(a.baseline))
for site in set(''.join(set(a.baseline))):
    b = a[a.baseline.str.contains(site)]
    for (bl, rows) in b.groupby('baseline'):
        i0 = rows.snr < 50
        i1 = rows.snr >= 50
        bl = bl if bl[1] == site else ''.join(bl[::-1])
        h = plt.errorbar(rows[i1].scan_no, rows[i1].coh, yerr=1./rows[i1].snr,
                         fmt='.', label='_nolegend_' if i1.sum() == 0 else bl, zorder=10)
        _ = plt.errorbar(rows[i0].scan_no, rows[i0].coh, yerr=1./rows[i0].snr,
                         fmt='.', label=bl if (i1.sum() == 0 and i0.sum() > 0) else '_nolegend_', color=h[0].get_color(), alpha=0.5)
    multline(elines)
    _ = plt.title('Coherence loss in 30 seconds [%.0f MHz]' % (b.iloc[0].ref_freq))
    tightx()
    _ = plt.xlim(0, plt.xlim()[1]*1.1)
    _ = plt.grid(axis='y', alpha=0.25)
    _ = plt.legend(loc='best')
    outliers = b[(b.coh < 0.8) & (b.snr > 20)]
    if len(outliers) > 0:
        _ = plt.plot(outliers.scan_no, outliers.coh,
                     'ko', ms=8, mfc='none', mew=2, zorder=-100)
    wide(12, 4)
    plt.show()
    outliers["expt_no scan_id source baseline snr coh".split()]